# Module 1: Agenda Processing
- Step 1: Downloading agendas .pdf files
- Step 2: Extracting text from agenda .pdf files
- Step 3: Segment agenda text into topic segments
- Step 4: Fetch links to legislation
- Step 5: Follow legislation links to obtain legislative text
- Step 6: Match legislative text to agenda topic
- Step 7: MANUAL matching of expert-written headlines to agenda topics

Specify the week of meetings to process.

In [ ]:
#### EDIT THIS
MONDAY_DATE = "YYYYMMDD"
FRIDAY_DATE = "YYYYMMDD"
#### EDIT THIS

In [ ]:
from datetime import datetime
WEEK = (MONDAY_DATE, FRIDAY_DATE)
START_DATE = datetime.strptime(WEEK[0], "%Y%m%d")
END_DATE = datetime.strptime(WEEK[1], "%Y%m%d")

## Step 1: Downloading agenda files from official website URLs

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pathlib import Path
from datetime import datetime

The folder `___input/agenda_url/` should contain a `.txt` file for each city council meeting. The `.txt` file should contain only the Legistar URL of the specific meeting's agenda. Each `.txt` file should be named such that the first 8 characters are the meeting date in YYYYMMDD format.

In [ ]:
INPUT_AGENDA_PATH = Path("../___input/agenda_url/")
assert INPUT_AGENDA_PATH.exists()

The agenda `.pdf` files will be saved to `_interim/agenda_raw/`. 

In [ ]:
RAW_AGENDA_PATH = Path("../_interim/agenda_raw/")
RAW_AGENDA_PATH.mkdir(parents=True, exist_ok=True)

Download agendas.

In [ ]:
for txt_file in INPUT_AGENDA_PATH.glob("*.txt"):

    # check if meeting took place in specified week
    date_str = txt_file.stem[:8]
    try:
        file_date = datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        # skip, meeting does not valid date
        continue
    if file_date < START_DATE or file_date > END_DATE:
        # skip, meeting did not take place in specified week
        continue


    # open .txt file
    with open(txt_file, "r", encoding="utf-8") as f:
        page_url = f.read().strip()

        
    # extract URL from file
    if not page_url:
        print(f"!!!! ISSUE WITH URL IN FILE: {txt_file}")
        continue


    # load url
    try:
        response = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
    except Exception as e:
        print(f"!!!! WEBSITE FAIL: {page_url}")
        continue


    # find .pdf file
    soup = BeautifulSoup(response.text, "html.parser")
    HTML_AGENDA_ID = "ctl00_ContentPlaceHolder1_hypMinutes"
    link_tag = soup.find("a", id=HTML_AGENDA_ID)
    if not link_tag or not link_tag.get("href"):
        print(f"!!! PDF NOT FOUND: {page_url}")
        continue
    pdf_url = urljoin(page_url, link_tag["href"])


    # download .pdf file
    try:
        pdf_response = requests.get(pdf_url, headers={"User-Agent": "Mozilla/5.0"})
        pdf_response.raise_for_status()
    except Exception as e:
        print(f"!!! PDF DOWNLOAD FAIL: {pdf_url}")
        continue

    # save to destination
    output_filename = RAW_AGENDA_PATH / (txt_file.stem + ".pdf")
    with open(output_filename, "wb") as f:
        f.write(pdf_response.content)

## Step 2: Extract text from agenda .pdf files

In [ ]:
import pdfplumber
from pathlib import Path

The folder `_interim/agenda_raw/` should now contain a .pdf for each meeting agenda. The next step is to extract the text from these .pdf files and save to a .txt file in the folder `_interim/agenda_processed/`.

In [ ]:
assert RAW_AGENDA_PATH.exists()
PROCESSED_AGENDA_PATH = Path("../_interim/agenda_processed/")
PROCESSED_AGENDA_PATH.mkdir(parents=True, exist_ok=True)

Utilize PDFplumber library to extract text.

In [ ]:
for pdf_file in RAW_AGENDA_PATH.glob("*.pdf"):

    # check if meeting took place in specified week
    date_str = txt_file.stem[:8]
    try:
        file_date = datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        # skip, meeting does not valid date
        continue
    if file_date < START_DATE or file_date > END_DATE:
        # skip, meeting did not take place in specified week
        continue


    # use PDFplumber to extract text
    with pdfplumber.open(pdf_file) as pdf:
        all_text = ""
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                all_text += page_text + "\n\n"

    output_path = PROCESSED_AGENDA_PATH / f"{pdf_file.stem}.txt"
    output_path.write_text(all_text, encoding="utf-8")

## Step 3: Segment agenda texts into topic segments

In [ ]:
import anthropic
import json
import csv
from pathlib import Path
import os

The agenda is segmented using Claude into topic segments and are saved as a `.csv` file in the `_interim/agenda_segments/` folder. First, load the Claude API key and client.

In [ ]:
CLAUDE_API_KEY = os.getenv("CLAUDE_KEY")
client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

assert PROCESSED_AGENDA_PATH.exists()
AGENDA_SEGMENTS_PATH = Path("../_interim/agenda_segments/")
AGENDA_SEGMENTS_PATH.mkdir(parents=True, exist_ok=True)

Function to build Claude prompt from agenda text. Claude should return a JSON parsable string.

In [ ]:
def build_agenda_segmentation_prompt(agenda_text: str) -> str:
    return f"""
You are a professional city council meeting assistant.

Given the full raw text of a meeting agenda, segment it into distinct agenda items. For each item:

1. Include the full agenda item title (e.g., "Bill 2023-114: Amending the zoning regulations").
2. Each **bill, paper, resolution, or ordinance** (e.g., "ORD. 2023-114", "RES. 2023-R016", "PAPER #412") counts as a **separate agenda item**, even if multiple items fall under the same section.
3. If a bill number or ordinance number appears, include it as part of the agenda item title.
4. Under each agenda item, include **all the text** that falls under it until the next agenda item begins.
5. Keep the original wording and formatting. Do not summarize or shorten the text.
6. Do not skip or omit any part of the agenda. This includes routine items such as “Roll Call,” “Public Comment," and other procedural sections.

Return the segmented agenda as a JSON array of strings. Each string is one agenda item with its full text.

**Important:** Return **only** the JSON array of strings with no additional text, explanation, or commentary. The output must be a valid JSON array.

[
  "[Agenda item title]\\n   [Full text under the item]",
  "[Next agenda item]\\n   [Full text under the item]",
  ...
]

Agenda:
\"\"\"{agenda_text}\"\"\"
"""

Prompt Claude and parse the string as JSON.

In [ ]:
for file_path in PROCESSED_AGENDA_PATH.glob("*.txt"):

    # check if meeting took place in specified week
    date_str = file_path.stem[:8]
    try:
        file_date = datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        # skip, meeting does not valid date
        continue
    if file_date < START_DATE or file_date > END_DATE:
        # skip, meeting did not take place in specified week
        continue

    # read processed agenda .txt file
    file_stem = file_path.stem
    output_path = AGENDA_SEGMENTS_PATH / f"{file_stem}.csv"
    text = file_path.read_text(encoding='utf-8')


    # build prompt and obtain JSON string
    prompt = build_agenda_segmentation_prompt(text)
    response = client.messages.create(
        model="claude-3-5-haiku-latest",
        max_tokens=8192,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    json_string = response.content[0].text.strip()


    # try to parse the JSON string and save it if parsable
    try:
        segments = json.loads(json_string)
        with output_path.open("w", newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["agenda_segment"])
            for segment in segments:
                writer.writerow([segment])
    except json.JSONDecodeError as e:
        print(f"!!!! LLM OUTPUT NOT JSON PARSABLE")

## Step 4: Fetching legislation links for each agenda segment

In [ ]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from urllib.parse import urljoin
import csv
import re

Finds all links to official legislation text for each meeting and stores as a `.csv` file in the `../_interim/legislation/` folder.

In [ ]:
assert INPUT_AGENDA_PATH.exists()
LEGISLATION_PATH = Path("../_interim/legislation/")
LEGISLATION_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
HTML_LEGISLATION_TABLE_ID = "ctl00_ContentPlaceHolder1_gridMain_ctl00"
for txt_file in INPUT_AGENDA_PATH.glob("*.txt"):

    # check if meeting took place in specified week
    date_str = txt_file.stem[:8]
    try:
        file_date = datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        # skip, meeting does not valid date
        continue
    if file_date < START_DATE or file_date > END_DATE:
        # skip, meeting did not take place in specified week
        continue
    

    # extract url from .txt file
    with open(txt_file, "r", encoding="utf-8") as f:
        page_url = f.read().strip()
    if not page_url:
        print(f"!!!! ISSUE WITH URL IN FILE: {txt_file}")
        continue


    # load webpage
    try:
        response = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
    except Exception as e:
        print(f"!!! WEBPAGE FAIL: {page_url}")
        continue


    # extract table from webpage using the html id
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", id=HTML_LEGISLATION_TABLE_ID)
    if not table:
        print(f"!!!! NO TABLE WITH ID '{HTML_LEGISLATION_TABLE_ID}' FOUND.")
        continue



    # extract and save URLs of legislation and their corresponding item
    output_csv_path = LEGISLATION_PATH / f"{txt_file.stem}.csv"
    with open(output_csv_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["item", "link"])


        # iterate table rows
        for row in table.find_all("tr"):
            cells = row.find_all("td")
            if not cells:
                continue

            first_col = cells[0]
            text = re.sub(r"[a-zA-Z\s]", "", first_col.get_text(strip=True))
            links = first_col.find_all("a", href=True)

            for a in links:
                href = urljoin(page_url, a["href"])
                if text and href:
                    writer.writerow([text, href])

## Step 5: Follow links to legislation and record the legislative text

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.firefox.options import Options
from pathlib import Path
import time

Follow the links to each legislation and record the legislative text in the agenda segments file.

In [ ]:
assert LEGISLATION_PATH.exists()

In [ ]:
firefox_options = Options()
firefox_options.add_argument("--width=1920")
firefox_options.add_argument("--height=1080")

driver = webdriver.Firefox(options=firefox_options)

In [ ]:
HTML_LEGISLATION_TAB_XPATH = '//li[contains(@class, "rtsLI") and contains(@class, "rtsLast")]//span[contains(@class, "rtsTxt") and text()="Text"]/ancestor::li'
HTML_LEGISLATION_TEXT_ID = "ctl00_ContentPlaceHolder1_pageText"

try:
    csv_files = list(LEGISLATION_PATH.glob("*.csv"))
    for csv_file in csv_files:

        # check if meeting took place in specified week
        date_str = csv_file.stem[:8]
        try:
            file_date = datetime.strptime(date_str, "%Y%m%d")
        except ValueError:
            # skip, meeting does not valid date
            continue
        if file_date < START_DATE or file_date > END_DATE:
            # skip, meeting did not take place in specified week
            continue


        df = pd.read_csv(csv_file)
        if "link" not in df.columns:
            print(f"!!! DID NOT DO STEP 4: {csv_file}.")
            continue



        texts = []
        for idx, url in enumerate(df["link"]):
            try:
                driver.get(url)
                wait = WebDriverWait(driver, 5)

                # click the "Text" tab
                tab_element = wait.until(EC.element_to_be_clickable((By.XPATH, HTML_LEGISLATION_TAB_XPATH)))
                tab_element.click()


                # check and access element containing text
                content_div = wait.until(EC.visibility_of_element_located((By.ID, HTML_LEGISLATION_TEXT_ID)))
                text_content = content_div.text.strip()

                # check for "Click here for full text" link
                try:
                    full_text_link = content_div.find_element(By.LINK_TEXT, "Click here for full text")
                    if full_text_link:
                        full_text_url = full_text_link.get_attribute("href")
                        print(f"    Found full text link, navigating to {full_text_url}")
                        driver.get(full_text_url)

            
                        content_div = wait.until(EC.visibility_of_element_located((By.ID, HTML_LEGISLATION_TEXT_ID)))
                        text_content = content_div.text.strip()
                except (TimeoutException, NoSuchElementException):
                    # text not long enough to have "Click here for full text" link
                    pass

                print(f"retrieved text length: {len(text_content)}")
                texts.append(text_content)

            except Exception:
                print(f"!!! ERROR PROCESSING URL: {url}")

                # default value
                texts.append("NO_LEGISLATION")

            time.sleep(1) 


        df["text"] = texts
        df.to_csv(csv_file, index=False)

finally:
    driver.quit()

## Step 6: Match legislation texts to agenda segments

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
assert AGENDA_SEGMENTS_PATH.exists()
assert LEGISLATION_PATH.exists()

Using the agenda item, match the legislation text to the correct agenda topic and add the legislatition text to the agenda segments file.

In [ ]:
for leg_file in LEGISLATION_PATH.glob("*.csv"):

    # check if meeting took place in specified week
    date_str = leg_file.stem[:8]
    try:
        file_date = datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        # skip, meeting does not valid date
        continue
    if file_date < START_DATE or file_date > END_DATE:
        # skip, meeting did not take place in specified week
        continue



    aseg_file = AGENDA_SEGMENTS_PATH / leg_file.name
    
    # read .csv files
    aseg_df = pd.read_csv(aseg_file)
    leg_df = pd.read_csv(leg_file)

    # default value
    aseg_df["matched_legislation"] = "NO_LEGISLATION"

    # find matches
    for _, leg_row in leg_df.iterrows():
        for aseg_idx, aseg_row in aseg_df.iterrows():
            if leg_row["item"] in aseg_row["agenda_segment"]:
                # set to legislation text if default value and append if already been changed
                if aseg_df.at[aseg_idx, "matched_legislation"] == "NO_LEGISLATION":
                    aseg_df.at[aseg_idx,"matched_legislation"] = leg_row["text"]
                else:
                    aseg_df.at[aseg_idx,"matched_legislation"] = aseg_df.at[aseg_idx,"matched_legislation"] + leg_row["text"]
        

    aseg_df.to_csv(aseg_file,index=False)

## Step 7: MANUAL matching of expert written headlines to agenda topics

In [ ]:
import pandas as pd

Manually align the expert written headlines to the agenda topic that is covered by the headline.

In [ ]:
df = pd.read_csv("../_interim/agenda_segments/20250401_REG.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[23, "true_headline"] = """Open-Ended Professional Services Contracts Add 27 Unspecified New Vendors"""
df.loc[27, "true_headline"] = """Portable Restroom Back Payment Sparks Inquiries into the Prevalence of Closed Public Restrooms in City Parks"""
df.to_csv("../_interim/agenda_segments/20250401_REG.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250402_STA.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[12, "true_headline"] = """Council to Approve Funding for Recruitment Video Aimed at Addressing Police Shortages"""
df.loc[17, "true_headline"] = """14 Acre Bakery Square Expansion Moves Forward with Two Key Amendments Ready for Public Hearing"""
df.to_csv("../_interim/agenda_segments/20250402_STA.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250408_POS.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[0, "true_headline"] = """Mayor's Vision Zero Plan Appears on Track, Still Awaiting Traffic Fatality Data"""
df.to_csv("../_interim/agenda_segments/20250408_POS.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250408_REG.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[8, "true_headline"] = """Council Seeks State Assistance in Eliminating the City’s Litter and Illegal Dumpsites"""
df.loc[10, "true_headline"] = """Untested Sexual Assault Kits to Be Processed"""
df.loc[32, "true_headline"] = """Council to Approve Funding for Recruitment Video Aimed at Addressing Police Shortages"""
df.to_csv("../_interim/agenda_segments/20250408_REG.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250409_STA.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[13, "true_headline"] = """City Receives State Funds for New Lighting Along Penn Ave in East Liberty"""
df.to_csv("../_interim/agenda_segments/20250409_STA.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250415_REG.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[8, "true_headline"] = """$40,000 Contract with Law Firm for “Immigration Matters” Authorized"""
df.loc[22, "true_headline"] = """City Receives State Funds for New Lighting Along Penn Ave in East Liberty"""
df.to_csv("../_interim/agenda_segments/20250415_REG.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250416_STA.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[2, "true_headline"] = """Council Reopens Discussion of the Process to Purchase City Property After a Rescinded Sale"""
df.loc[9, "true_headline"] = """Untested Sexual Assault Kits to Be Processed"""
df.loc[14, "true_headline"] = """Proposed July 2024 Contract for Shelter Services Held Another Five Weeks"""
df.to_csv("../_interim/agenda_segments/20250416_STA.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250422_POS.csv")
df["true_headline"] = "NO_HEADLINE"
df.to_csv("../_interim/agenda_segments/20250422_POS.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250422_REG.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[18, "true_headline"] = """Riverlife Barge Venue to Dock at Allegheny Landing Park"""
df.loc[19, "true_headline"] = """Bill to Allow Pollinator Gardens and Natural Landscapes Introduced"""
df.loc[21, "true_headline"] = """Challenged Historic Designation for Former Gay Bar to Receive Public Hearing"""
df.loc[38, "true_headline"] = """Untested Sexual Assault Kits to Be Processed"""
df.to_csv("../_interim/agenda_segments/20250422_REG.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250423_PUB.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[1, "true_headline"] = """'Tiny Lots' Approved After Spirited Debate"""
df.to_csv("../_interim/agenda_segments/20250423_PUB.csv", index=False)


df = pd.read_csv("../_interim/agenda_segments/20250423_STA.csv")
df["true_headline"] = "NO_HEADLINE"
df.loc[5, "true_headline"] = """$40,000 Contract with Law Firm for “Immigration Matters” Authorized"""
df.to_csv("../_interim/agenda_segments/20250423_STA.csv", index=False)

This concludes Module 1: Agenda Processing.